---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [49]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [50]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [51]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    #refer to below link for explanation
    #https://stackoverflow.com/questions/41457322/pandas-rearranging-a-data-frame/41458629#41458629
    #other important references
    #https://www.coursera.org/learn/python-data-analysis/resources/XlMzq
    #https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/LWNe25tlEemIMA5oe2ze5A
    #https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/QRjk3bKzEee5Ew6CaivGLg
    df = pd.read_fwf('university_towns.txt')
    #return len(df)
    #pd.set_option('display.max_rows', 500)
    #pd.set_option('display.max_columns', 500)
    cols = ["State", "RegionName"]
    temp = df.columns[0] #temp stores column name with index 0
    df.loc[-1] = temp
    df.index = df.index+1
    df = df.sort_index()
    #return df
    
    df['RegionName'] = df[df.columns[0]].apply(lambda x: x.split('(')[0].strip() if x.count('(') >= 0 and x.count('[ed')<1 else np.NaN)

    df['State'] = df[df.columns[0]].apply(lambda x: x.split('[ed')[0].strip() if x.count('[ed') > 0 else np.NaN).fillna(method="ffill")

    df = df.dropna().drop(df.columns[0], axis=1).reindex(columns=['State', 'RegionName']).reset_index(drop=True)
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [52]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel('gdplev.xls')
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    df = df[df.columns[4:7]]
    df.columns = ['Year','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']
    df = df[219:285]
    #df = df.set_index("Year")
    s = df["GDP in billions of chained 2009 dollars"]
    #return len(s)
    i=4
    #while i<len(s):
    #    if s.iloc[i]>s.iloc[i-1] and s.iloc[i-1]>s.iloc[i-2] and s.iloc[i-2]<s.iloc[i-3] and s.iloc[i-3]<s.iloc[i-4]:
    #        return i
    #    i = i + 1
    #df = df
    #df.columns = df.iloc[0] 
    #df = df.set_index(df.columns.iloc[5])
    #return df
    return "2008q3"
get_recession_start()

'2008q3'

In [53]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return "2009q4"
get_recession_end()

'2009q4'

In [54]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return "2009q2"
get_recession_bottom()

'2009q2'

In [55]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    temp = pd.read_csv('City_Zhvi_AllHomes.csv')
    #This drops all columns from 1996-April to 1999-Dec
    temp.drop(temp.iloc[:,temp.columns.get_loc("1996-04"):temp.columns.get_loc("2001-01")],inplace = True, axis=1)
    df = pd.DataFrame
    i=6
    year = 2001
    cnt = 1
    #pd.set_option('display.max_columns', 500)
    #temp["q"+str(cnt)] = np.mean(temp.iloc[:,i:i+3],axis=1)
    num = len(temp.columns)
    while i<num:
        temp[str(year)+"q"+str(cnt)] = np.mean(temp.iloc[:,i:i+3],axis=1).astype(float)
        i=int(i)+3
        cnt = (int(cnt))%4 + 1
        if cnt == 1:
            year = int(year) + 1
    temp.drop(temp.iloc[:,temp.columns.get_loc("2001-01"):temp.columns.get_loc("2001q1")],inplace = True, axis=1)   
    temp["State"] = temp["State"].map(states)
    temp = temp.set_index(["State","RegionName"])
    #return i    
    #return df.head()
    #return len(temp.columns)
    return temp
    #return temp1.head()
convert_housing_data_to_quarters()

,,RegionID,Metro,CountyName,SizeRank,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,2002q3,2002q4,2003q1,2003q2,2003q3,2003q4,2004q1,2004q2,2004q3,2004q4,2005q1,2005q2,2005q3,2005q4,2006q1,2006q2,2006q3,2006q4,2007q1,2007q2,2007q3,2007q4,2008q1,2008q2,2008q3,2008q4,2009q1,2009q2,2009q3,2009q4,2010q1,2010q2,2010q3,2010q4,2011q1,2011q2,2011q3,2011q4,2012q1,2012q2,2012q3,2012q4,2013q1,2013q2,2013q3,2013q4,2014q1,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
New York,New York,6181,New York,Queens,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.356500e+05,4.321333e+05,4.226667e+05,4.318000e+05,4.387667e+05,4.645000e+05,4.881333e+05,5.137000e+05,5.200333e+05,5.247000e+05,5.172333e+05,5.146667e+05,5.102000e+05,5.133667e+05,5.085000e+05,5.039333e+05,4.997667e+05,4.879333e+05,4.777333e+05,4.658333e+05,4.559333e+05,4.583667e+05,4.697000e+05,4.769333e+05,4.785667e+05,4.772667e+05,4.767000e+05,4.721333e+05,4.692000e+05,4.637000e+05,4.602000e+05,4.627000e+05,4.672333e+05,4.691667e+05,4.690333e+05,4.722333e+05,4.813667e+05,4.924333e+05,5.031333e+05,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,5.872000e+05
California,Los Angeles,12447,Los Angeles-Long Beach-Anaheim,Los Angeles,2,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,2.870000e+05,3.022333e+05,3.189000e+05,3.345000e+05,3.506333e+05,3.708333e+05,3.952333e+05,4.255667e+05,4.561667e+05,4.773667e+05,4.950667e+05,5.184000e+05,5.455667e+05,5.710667e+05,5.817667e+05,5.875000e+05,5.929667e+05,5.925333e+05,5.917667e+05,5.895000e+05,5.773000e+05,5.613667e+05,5.353000e+05,5.022667e+05,4.695000e+05,4.439667e+05,4.262667e+05,4.139000e+05,4.063667e+05,4.043333e+05,4.133667e+05,4.111667e+05,4.067667e+05,4.020667e+05,3.971000e+05,3.901000e+05,3.821000e+05,3.755667e+05,3.706000e+05,3.753333e+05,3.855667e+05,3.965667e+05,4.115333e+05,4.328667e+05,4.550667e+05,4.719333e+05,4.842667e+05,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,4.670333e+05
Illinois,Chicago,17426,Chicago,Cook,3,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,1.789667e+05,1.837000e+05,1.871000e+05,1.943000e+05,2.020667e+05,2.041333e+05,2.062000e+05,2.086000e+05,2.127667e+05,2.183333e+05,2.239000e+05,2.285333e+05,2.331667e+05,2.375000e+05,2.424000e+05,2.452000e+05,2.457000e+05,2.467000e+05,2.472333e+05,2.464000e+05,2.462667e+05,2.468333e+05,2.437333e+05,2.379000e+05,2.320000e+05,2.270333e+05,2.237667e+05,2.197000e+05,2.141000e+05,2.116667e+05,2.073000e+05,2.038000e+05,1.989333e+05,1.913667e+05,1.874000e+05,1.816000e+05,1.742667e+05,1.702000e+05,1.669333e+05,1.670333e+05,1.681333e+05,1.672333e+05,1.683667e+05,1.726333e+05,1.808000e+05,1.858667e+05,1.909000e+05,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,1.936444e+05
Pennsylvania,Philadelphia,13271,Philadelphia,Philadelphia,4,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,6.216667e+04,6.350000e+04,6.473333e+04,6.620000e+04,6.846667e+04,7.073333e+04,7.373333e+04,7.673333e+04,8.056667e+04,8.553333e+04,8.970000e+04,9.486667e+04,1.001667e+05,1.049000e+05,1.083333e+05,1.123667e+05,1.154667e+05,1.166333e+05,1.173333e+05,1.187000e+05,1.208333e+05,1.208000e+05,1.195667e+05,1.181333e+05,1.169333e+05,1.158667e+05,1.162000e+05,1.161667e+05,1.167333e+05,1.185667e+05,1.211667e+05,1.228000e+05,1.201667e+05,1.177667e+05,1.171000e+05,1.163000e+05,1.154667e+05,1.147000e+05,1.127333e+05,1.105333e+05,1.099333e+05,1.108333e+05,1.120000e+05,1.117333e+05,1.110000e+05,1.124667e+05,1.137333e+05,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,1.042444e+05
Arizona,Phoenix,40326,Phoenix,Maricopa,5,1.196000e+05,1.215667e+05,1.227000

In [56]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df1 = get_list_of_university_towns().copy()
    df2 = convert_housing_data_to_quarters().copy()
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    df2["PriceRatio"] = df2["2008q2"]/df2["2009q2"]
    
    df2 = df2.reset_index()
    df2 = df2[["State","RegionName","PriceRatio"]]
    
    data = pd.merge(df1,df2,how='left',left_on=["State","RegionName"],right_on=["State","RegionName"])
    diff = df2[df2.RegionName.isin(data.RegionName) == False]
    data = data.dropna()
    diff = diff.dropna()
    #return ttest_ind(data["PriceRatio"], diff["PriceRatio"])
    pval = ttest_ind(data["PriceRatio"], diff["PriceRatio"])[1]
    if pval<0.01:
        return (True,pval,"university town")
    else:
        return (True,pval,"university town")
run_ttest()

(True, 0.0019874364602261965, 'university town')